# Llama 3-8B Instruct model prompting

In this series of notebooks, I will develop an NBA AI agent using **LLaMA 3-8B Instruct** as the foundational model. The agent's primary function is to generate accurate and efficient **SQLite queries** to extract insights from the `nba_roster` database. Given a structured schema and a natural language question, the agent translates the request into an executable SQL statement.  

### **Pipeline Overview:**  
1. The user submits a question in natural language.  
2. The model processes the question alongside relevant instructions (prompt + database schema).  
3. The model generates an SQL query.  
4. The query is executed against the database.  
5. The retrieved data is returned to the user.  

### **Objective of this notebook:**  
- Construct effective prompts for the foundational model.  
- Analyze and refine the model’s output structure.

This project includes a set of predefined classes specifically designed to streamline the development of our NBA agent. These classes handle model interactions, prompt management, schema retrieval, and SQL query extraction. They are imported as follows:

In [1]:
from src.llm.llm_model import LLMModel
from src.llm.llm_prompt import PromptBuilder
from src.utils.schema_provider import SchemaProvider
from src.llm.sql_extractor import SQLQueryExtractor

/home/stefany/interview_Material/interviews/interview_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's start loading and prompting our Llama-3-8B-Instruct model, which is gonna be our foundational model for this project. 

In [2]:
llm = LLMModel(model_name="llama-3b-8B/Meta-Llama-3-8B-Instruct")

INFO:src.llm.llm_model:Releasing GPU memory...
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.17s/it]
INFO:src.llm.llm_model:Model loaded from: models/llama-3-8B
INFO:src.llm.llm_model:Model successfully loaded on device: cuda:0


### **Prompting the model:** 
Before writing a prompt, ask yourself:

- What do I want the model to do?
- What kind of response am I expecting (e.g., SQL query, summary, structured text)?
- How specific does the output need to be?

To achieve our goal of creating an effective SQL query agent, we must guide the model to generate structured and accurate outputs by embedding clear and comprehensive instructions within our prompt. Our agent prompt will follow this structured approach:

- **System instructions:** Define the model’s role explicitly, such as an expert SQL analyst, ensuring it understands its function and scope.
- **Database schema or structure:**  Since the model lacks direct access to the database and has no prior knowledge of its structure or contents, we must provide this information explicitly. This includes table names, column definitions, data types, and relevant relationships to enable the model to construct valid and contextually appropriate SQL queries.  
- **Expected output:** Specify the required structure for the model’s response, ensuring consistency and usability. This may include formatting the SQL query in a structured block, adding explanatory comments, or providing alternative query optimizations if applicable.
- **User input:** Since our agent goal is to generate a SQL query given a natural language question, we need to included this questions in our prompt to guide the model.

Let's start with a simple question, where we want to know the following information contained in the database:

In [3]:
question = "Who is the highest paid NBA player?"

In [4]:
system_instructions = "You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:\n{schema}\n\nWrite a SQL query to answer the following question. Follow instructions exactly."

In [5]:
prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{system_instructions}<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{question}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n"

The system instructions and prompt format are fixed (it may change depending of your application), and we only insert the database schema and the natural language question.

In [6]:
variables = {
    "question": question,
    "schema": SchemaProvider.get_nba_schema_1(),
}
print(SchemaProvider.get_nba_schema_1()) # Let's load the database schema. 

        0|Team|TEXT 
        1|NAME|TEXT  
        2|Jersey|TEXT 
        3|POS|TEXT
        4|AGE|INT 
        5|HT|TEXT 
        6|WT|TEXT 
        7|COLLEGE|TEXT 
        8|SALARY|TEXT
        


In [7]:
prompt_builder = PromptBuilder(
    system_instructions=system_instructions,
    prompt=prompt
)

# Generate final prompt
prompt_builder.add_variables(variables)
formatted_prompt = prompt_builder.build_prompt()
print(formatted_prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
        0|Team|TEXT 
        1|NAME|TEXT  
        2|Jersey|TEXT 
        3|POS|TEXT
        4|AGE|INT 
        5|HT|TEXT 
        6|WT|TEXT 
        7|COLLEGE|TEXT 
        8|SALARY|TEXT
        

Write a SQL query to answer the following question. Follow instructions exactly.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Who is the highest paid NBA player?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>



In [8]:
response = llm.generate_response(formatted_prompt)
print(response) # finally, we get the answer from the model. 

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


To answer this question, we can use the following SQL query:

```sql
SELECT NAME, SALARY
FROM nba_roster
ORDER BY SALARY DESC
LIMIT 1;
```

This query will return the name and salary of the highest paid NBA player. The `ORDER BY SALARY DESC` clause sorts the players by their salary in descending order (highest to lowest), and the `LIMIT 1` clause limits the result to the top 1 row, which is the highest paid player.


Before moving forward, there are two important concepts to note here:

- When prompting a model, it is essential to consider its specifications to ensure optimal performance. Some models require special tokens (e.g., \<system>, \<user>, \<assistant>) to structure the prompt correctly. Before crafting a prompt, review the model’s documentation to determine whether such tokens are necessary.

    For instance, LLaMA 3 uses special tokens to indicate different sections of the prompt. The system instructions are marked with: <|begin_of_text|><|start_header_id|>system<|end_header_id|>. User input follows the same format: <|begin_of_text|><|start_header_id|>user<|end_header_id|>. Finally, the end of the prompt, where the model is expected to respond, should be explicitly marked as: <|begin_of_text|><|start_header_id|>assistant<|end_header_id|>.

- The model's output consists of both the generated SQL query and a natural language explanation, rather than structured JSON data. This behavior has important implications for app development:  
  - Generating more output tokens increases costs, latency, and can reduce app performance.  
  - A method is needed to convert natural language into a structured format.


Let's refine the prompt so that it outputs only the SQL query. 

In [9]:
system_instructions_2 = """You are an experienced NBA analyst with 15 years of expertise in writing complex SQL queries. 
Consider the `nba_roster` table with the following schema: {schema}\n\nWrite a **SQL query** to answer the question below.

**Output only the SQL query**—do not include any explanations or additional text. Follow the instructions exactly."""

prompt_builder = PromptBuilder(
    system_instructions=system_instructions_2, # Let's change the system instructions, and check the output of the model.
    prompt=prompt
)

# Generate final prompt
prompt_builder.add_variables(variables)
formatted_prompt = prompt_builder.build_prompt()
print(formatted_prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an experienced NBA analyst with 15 years of expertise in writing complex SQL queries. 
Consider the `nba_roster` table with the following schema:         0|Team|TEXT 
        1|NAME|TEXT  
        2|Jersey|TEXT 
        3|POS|TEXT
        4|AGE|INT 
        5|HT|TEXT 
        6|WT|TEXT 
        7|COLLEGE|TEXT 
        8|SALARY|TEXT
        

Write a **SQL query** to answer the question below.

**Output only the SQL query**—do not include any explanations or additional text. Follow the instructions exactly.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Who is the highest paid NBA player?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>



In [10]:
response = llm.generate_response(formatted_prompt)
print(response) # Output the generated response

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


SELECT * FROM nba_roster WHERE SALARY = (SELECT MAX(SALARY) FROM nba_roster);


In our next iterative process, we will improve the schema for our database, including examples of the types of data it contains.

In [11]:
variables = {
    "question": question,
    "schema": SchemaProvider.get_nba_schema_2(),
}
print(SchemaProvider.get_nba_schema_2())


        0|Team|TEXT eg. "Toronto Raptors"
        1|NAME|TEXT eg. "Otto Porter Jr."
        2|Jersey|TEXT eg. "0" and when null has a value "NA"
        3|POS|TEXT eg. "PF"
        4|AGE|INT eg. "22" in years
        5|HT|TEXT eg. `6' 7"` or `6' 10"`
        6|WT|TEXT eg. "232 lbs"
        7|COLLEGE|TEXT eg. "Michigan" and when null has a value "--"
        8|SALARY|TEXT eg. "$9,945,830" and when null has a value "--"
        


In [12]:
prompt_builder = PromptBuilder(
    system_instructions=system_instructions_2, # Let's change the system instructions, and check the output of the model.
    prompt=prompt
)

# Generate final prompt
prompt_builder.add_variables(variables)
formatted_prompt = prompt_builder.build_prompt()
print(formatted_prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an experienced NBA analyst with 15 years of expertise in writing complex SQL queries. 
Consider the `nba_roster` table with the following schema: 
        0|Team|TEXT eg. "Toronto Raptors"
        1|NAME|TEXT eg. "Otto Porter Jr."
        2|Jersey|TEXT eg. "0" and when null has a value "NA"
        3|POS|TEXT eg. "PF"
        4|AGE|INT eg. "22" in years
        5|HT|TEXT eg. `6' 7"` or `6' 10"`
        6|WT|TEXT eg. "232 lbs"
        7|COLLEGE|TEXT eg. "Michigan" and when null has a value "--"
        8|SALARY|TEXT eg. "$9,945,830" and when null has a value "--"
        

Write a **SQL query** to answer the question below.

**Output only the SQL query**—do not include any explanations or additional text. Follow the instructions exactly.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Who is the highest paid NBA player?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>



In [13]:
response = llm.generate_response(formatted_prompt)
print(response) # Output the generated response

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


SELECT NAME, SALARY
FROM nba_roster
WHERE SALARY!= '--'
ORDER BY CAST(SALARY AS INTEGER) DESC
LIMIT 1;


## How to return structured data from a model

In [15]:
print(type(response)) # Let's go back to this concept. LLMs output text. So how can we get an structured output? 

<class 'str'>


Structured outputs from LLMs are essential for applications requiring data in specific formats, such as JSON or XML. Structuring the output of the model enhances accuracy and consistency in production pipelines, making the data more reliable for downstreams processes. Some common techniques for generating structured outputs from LLMs: 

- **Prompt engineering:** Instruct the model to produce outputs in a desire structure.
- **Output parsing:** Parsing the natural language response into structured formats using tools like regular expressions or specialized parsers.
- **Schema-guided Generation:** Defining a schema that the LLM adheres to during output generation, ensuring the responses conform to a predefined structure.

For our agent, I will implement a combination of prompting techniques and parsing methods. I will enforce the SQL query output using Pydantic. Using a Pydantic model ensures a valid SQL query is returned.

Next steps: 
- Define a Pydantic schema (SQLQuery) to enforce structure.
- Initialize the LLM.
- Use PydanticOutputParser to ensure our model follows the schema.
- Included the output format instructions in the prompt.
- Retrieve the response from the model.
- Parse the response into a SQLQuery object. If the response is malformed (e.g., missing a field), Pydantic will raise an error.


In [24]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

# Define Pydantic model.
class SQLQuery(BaseModel): 
    query: str = Field(..., description="The SQL query string")

# We modify the prompt such as we include the format instructions. 
system_instructions = """You are an experienced NBAparser = PydanticOutputParser(pydantic_object=SQLQuery)
 analyst with 15 years of expertise in writing complex SQL queries. 
Consider the `nba_roster` table with the following schema: {schema}\n\nWrite a **SQL query** to answer the question below.

**Output only the SQL query**—do not include any explanations or additional text. Follow the instructions exactly. {format_instructions}"""

prompt_builder = PromptBuilder(
    system_instructions=system_instructions, # Let's change the system instructions with the output format instructions included.
    prompt=prompt
)

parser = PydanticOutputParser(pydantic_object=SQLQuery)

# Update variables to included format instructions: 
variables = {
    "question": question,
    "schema": SchemaProvider.get_nba_schema_2(),
    "format_instructions": parser.get_format_instructions()
}

prompt_builder.add_variables(variables)
formatted_prompt = prompt_builder.build_prompt()
print(formatted_prompt)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an experienced NBAparser = PydanticOutputParser(pydantic_object=SQLQuery)
 analyst with 15 years of expertise in writing complex SQL queries. 
Consider the `nba_roster` table with the following schema: 
        0|Team|TEXT eg. "Toronto Raptors"
        1|NAME|TEXT eg. "Otto Porter Jr."
        2|Jersey|TEXT eg. "0" and when null has a value "NA"
        3|POS|TEXT eg. "PF"
        4|AGE|INT eg. "22" in years
        5|HT|TEXT eg. `6' 7"` or `6' 10"`
        6|WT|TEXT eg. "232 lbs"
        7|COLLEGE|TEXT eg. "Michigan" and when null has a value "--"
        8|SALARY|TEXT eg. "$9,945,830" and when null has a value "--"
        

Write a **SQL query** to answer the question below.

**Output only the SQL query**—do not include any explanations or additional text. Follow the instructions exactly. The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"proper

In [26]:
response = llm.generate_response(formatted_prompt)
parsed_response = parser.parse(response) # Converts the raw LLM output into a validated SQLQuery object.

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


query="SELECT * FROM nba_roster WHERE SALARY!= '--' ORDER BY CAST(SUBSTRING(SALARY, 2) AS INTEGER) DESC LIMIT 1"
<class '__main__.SQLQuery'>
SELECT * FROM nba_roster WHERE SALARY!= '--' ORDER BY CAST(SUBSTRING(SALARY, 2) AS INTEGER) DESC LIMIT 1


In [27]:
print(parsed_response) 
print(type(parsed_response))
print(parsed_response.query)

query="SELECT * FROM nba_roster WHERE SALARY!= '--' ORDER BY CAST(SUBSTRING(SALARY, 2) AS INTEGER) DESC LIMIT 1"
<class '__main__.SQLQuery'>
SELECT * FROM nba_roster WHERE SALARY!= '--' ORDER BY CAST(SUBSTRING(SALARY, 2) AS INTEGER) DESC LIMIT 1
